In [1]:
import pandas as pd
from collections import Counter
import nltk
from nltk.corpus import stopwords
import numpy as np

In [2]:
df=pd.read_csv('edmunds5.csv')
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rachana/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/rachana/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
stop = stopwords.words('english')
def remove_stopwords(text):
    words = nltk.word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_words)
df['body']=df['body'].str.lower()

df['body'] = df['body'].astype(str).apply(remove_stopwords)

In [62]:
#####CSV method
mapping_df = pd.read_csv('car_models_and_brands.csv')

# Load the second CSV file (contains model names)
model_df = pd.read_csv('processed_file.csv')


# Create a dictionary from the mapping DataFrame
model_to_brand = dict(zip(mapping_df['Model'], mapping_df['Brand']))

# Function to replace model names with brand names
def replace_model_with_brand(model_name):
    return model_to_brand.get(model_name, model_name)

# Apply the replacement function to the 'Model' column in the second DataFrame
model_df['body'] = model_df['body'].apply(replace_model_with_brand)

# Save the updated DataFrame to a new CSV file
model_df.to_csv('updated_model_names.csv', index=False)

In [4]:
df['body']

0       braking - sorry 70 0 braking 189 reported c & ...
1       new 2004 accord drove driveway last night . go...
2       love numbers , compare performance price numbe...
3       kd , people buy tl accord , reason bought 330 ...
4       pg48477 ... prove point . luxury primary crite...
                              ...                        
4995    `` meaningless '' guess 's meaningless actuall...
4996    guess everyone hung whole msrp value thing . g...
4997    please stop yelling ! consider used camaro z28...
4998    response exepected ... discounting areas bmw e...
4999    `` please mountain , curvy , wavy road nonsens...
Name: body, Length: 5000, dtype: object

In [5]:
# Map the brand names to the car names
mapping_df = pd.read_csv('car_models_and_brands.csv')
model_to_brand = dict(zip(mapping_df['Model'], mapping_df['Brand']))
def replace_model_with_brand(model_name):
    return model_to_brand.get(model_name, model_name)

df['body'] = df['body'].str.lower().apply(replace_model_with_brand)


In [6]:
brands=mapping_df['Brand'].unique()
values_to_remove = ['car', 'seat', 'problem','"hyundai,"','hyundai.','"kia,"','kia.']

brands = [x for x in brands if x not in values_to_remove]

In [7]:
# Load the CSV file with brand names
#brand_df = pd.read_csv('car_models_and_brands.csv')

# Load the CSV file with the "body" column
#target_df = pd.read_csv('updated_model_names.csv')


# Initialize an empty dictionary to store brand frequencies
brand_freq = {}

# Iterate through the "body" column of the target DataFrame
for _, row in df.iterrows():
    body_text = row['body']
    # Check for NaN values and skip them
    if not isinstance(body_text, str) and np.isnan(body_text):
        continue
    
    # Split the "body" text into words
    words = body_text.split()
    
    words = [word.lower() for word in words] 
    words = list(set(words))

    # Count the frequency of brand names in the "body" text
    for word in words:
        if word in brands:
            brand_freq[word] = brand_freq.get(word, 0) + 1

# Now, brand_freq dictionary contains the frequency counts of brand names in the "body" column
print(brand_freq)

{'bmw': 1177, 'honda': 169, 'acura': 382, 'sedan': 528, 'hyundai': 66, 'pontiac': 23, 'mercedes': 125, 'audi': 262, 'mazda': 60, 'infiniti': 336, 'lincoln': 45, 'chrysler': 44, 'dodge': 26, 'cadillac': 104, 'toyota': 120, 'nissan': 164, 'subaru': 112, 'ford': 63, 'volvo': 110, 'volkswagen': 15, 'mitsubishi': 15, 'saturn': 10, 'buick': 22, 'mercedes-benz': 25, 'kia': 21, 'mercury': 5, 'suzuki': 8, 'volkwagen': 1, 'chevrolet': 4}


In [8]:
sorted_dict = sorted(brand_freq.items(), key=lambda item: item[1],reverse=True)
top_10_brands=sorted_dict[:10]
for brand, frequency in top_10_brands:
    print(f'{brand}: {frequency}')
top_10_list = [item[0] for item in top_10_brands]
print(top_10_list)

bmw: 1177
sedan: 528
acura: 382
infiniti: 336
audi: 262
honda: 169
nissan: 164
mercedes: 125
toyota: 120
subaru: 112
['bmw', 'sedan', 'acura', 'infiniti', 'audi', 'honda', 'nissan', 'mercedes', 'toyota', 'subaru']


In [11]:
# Initialize a dictionary to store brand mentions per post
brand_mentions_per_post = {brand: set() for brand in top_10_list}

# Initialize dictionaries to store brand co-mentions and individual brand mentions
co_mentions = {}
for brand1 in top_10_list:
    co_mentions[brand1] = {}
    for brand2 in top_10_list:
        co_mentions[brand1][brand2] = 0
individual_mentions = {brand: 0 for brand in top_10_list}

# Function to calculate lift ratio between two brands
def calculate_lift(brand1, brand2, total_posts):
    if brand1 == brand2:
        return 0  # Lift ratio between the same brand is 0
    # Calculate lift using the formula: lift(brand1, brand2) = (P(brand1 and brand2) / (P(brand1) * P(brand2))) * N
    p_brand1_and_brand2 = co_mentions[brand1][brand2]
    p_brand1 = individual_mentions[brand1]
    p_brand2 = individual_mentions[brand2]
    if p_brand1 == 0 or p_brand2 == 0:
        return 0  
    else:
        return (p_brand1_and_brand2 / (p_brand1 * p_brand2)) * total_posts

# Iterate through the "body" column of the target DataFrame
for _, row in df.iterrows():
    body_text = row['body']
    
    # Check for NaN values and skip them
    if not isinstance(body_text, str) and np.isnan(body_text):
        continue
    
    # Split the "body" text into words
    words = body_text.split()
    
    # Convert words to lowercase for case-insensitive matching
    words = [word.lower() for word in words]
    
    # Keep track of previously mentioned brands in the current post
    mentioned_brands_in_post = set()
    
    for i, word in enumerate(words):
        if word in top_10_list:
            # Check if the brand was already mentioned in the current post
            if word not in mentioned_brands_in_post:
                # Increment individual mention count for the brand
                individual_mentions[word] += 1
                # Update the set of mentioned brands in the current post
                mentioned_brands_in_post.add(word)
                
                # Check for co-mentions of other brands in the same post
                for j in range(i + 1, min(i + 6, len(words))):  # Change '6' to your desired separation limit
                    if words[j] in top_10_list:
                        # Increment co-mention count for the pair of brands
                        co_mentions[word][words[j]] += 1

# Calculate total number of posts
total_posts = len(df)

# Calculate lift ratios for brand pairs
lift_ratios = {}
for brand1 in top_10_list:
    for brand2 in top_10_list:
        if brand1 != brand2:
            lift_ratio = calculate_lift(brand1, brand2, total_posts)
            lift_ratios[(brand1, brand2)] = lift_ratio

# Sort lift ratios in descending order
sorted_lift_ratios = sorted(lift_ratios.items(), key=lambda x: x[1], reverse=True)

# Print the top 10 lift ratios and associated brand pairs
for (brand1, brand2), lift_ratio in sorted_lift_ratios[:10]:
    print(f'Lift({brand1}, {brand2}) = {lift_ratio:.2f}')

Lift(honda, toyota) = 5.18
Lift(toyota, honda) = 4.68
Lift(toyota, nissan) = 3.30
Lift(honda, nissan) = 2.71
Lift(nissan, toyota) = 2.03
Lift(nissan, honda) = 1.62
Lift(acura, infiniti) = 1.29
Lift(infiniti, acura) = 1.01
Lift(audi, mercedes) = 0.92
Lift(acura, audi) = 0.90
